# Building a deep convolutional neural network

You should run this notebook online in either [Kaggle](https://www.kaggle.com/) or [Google Colab](https://colab.research.google.com), as described on the *Running notebooks online* page for this module. When running the notebook online, you can access machines with a GPU, which will make training the networks a lot faster. The cell below tests if you are indeed running on a machine with a GPU.

In [ ]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

If everything is working correctly, you should see a single GPU listed as the device for this machine.

Google is offering quite an expensive resource to people for free, so there are some restrictions you should take into account. If you are inactive in the Colab for a while, you will get disconnected from the machine and it will be reassigned to someone else. If you use too much memory, either RAM or harddisk, you'll also get disconnected and have to reconnect to a new machine.

Note that any time you get disconnected, switch runtime or restart the runtime, you will have to reconnect to a new machine. When this happens the whole notebook will be restarted, so you'll have to run any code cells, that for example load the data, again. Having to restart and rerun the cells can be a bit annoying, but it is a small inconvenience for getting free access to a GPU.

## CIFAR-10 dataset

Speaking of loading the data, run the cell below to download the CIFAR-10 dataset on this machine.


In [ ]:
![ ! -d 'cifar-10-batches-py' ] && curl -O https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
![ ! -d 'cifar-10-batches-py' ] && tar -xzvf cifar-10-python.tar.gz

The CIFAR-10 dataset is a classic dataset for deep learning and was originally created by Alex Krizhevsky. Start by reading the description of the dataset here: https://www.cs.toronto.edu/~kriz/cifar.html

Take a look at the format of the images and the random samples from the 10 classes shown on the page there, and make sure you understand the classification task from the dataset before continuing.

Below is some code to further unpack and prepare these 60.000 images. There are a couple of important things done as part of loading the data:

* Each image is reshaped back to 32 x 32 pixels and 3 color channels using [np.reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html)
* Each of the labels is transformed into a *one-hot encoding* using the [to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) function.
* The 5 separate files of training data are merged to a single *Numpy* array using [np.vstack](https://numpy.org/doc/stable/reference/generated/numpy.vstack.html)

In [ ]:
import os
import pickle
import numpy as np

def unpickle(filename):
    with open(filename, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
        images = [img.reshape((32,32,3), order='F').transpose(1, 0, 2) for img in data[b'data']]

        labels = tf.keras.utils.to_categorical(data[b'labels'])
        return (np.array(images), labels)


train_images, train_labels = [np.vstack(tr) for tr in zip(*[
        unpickle(os.path.join('cifar-10-batches-py', f'data_batch_{i+1}')) for i in range(5)])]

test_images, test_labels = unpickle(os.path.join('cifar-10-batches-py', 'test_batch'))

## Model evaluation function

Next up we'll provide a function that you can use for training your neural networks. This function takes a neural network `model`, training and validation data, and does a couple of things:

* Compiles the model using
    * a [CategoricalCrossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy) loss function. This is the same classification error function we used in combination with the Softmax activation.
    * an additional metric of [Accuracy](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Accuracy), to see how well the predictions match the actual labels.
    * an [Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) optimizer, which is a more advanced version of the gradient descent algorithm we've used up to this point to optimize the weights.
* Creates a preprocessing [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) for the training and validation data.
* Fits the compiled model using the preprocessed training data for the specified number of *epochs*.
* Plots the loss and accuracy for the training and validation data for each epoch.
* Prints the final validation accuracy.

We'll dive more deeply into what the preprocessing steps can do exactly in later sections. For now, no preprocessing is applied. Make sure you understand what all the other steps do before moving on to the next section.

In [ ]:
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models, preprocessing

def train_and_evaluate(model, train_x, train_y, val_x, val_y, preprocess={}, epochs=20, augment={}):

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    train_gen = preprocessing.image.ImageDataGenerator(**preprocess, **augment)
    train_gen.fit(train_x)

    # Note: The validation preprocessor is created without any augmentations. See Q10 for the reason.
    val_gen = preprocessing.image.ImageDataGenerator(**preprocess)
    # Note: The validation preprocessor is fitted on the training data. See Q3 for the reason.
    val_gen.fit(train_x)

    history = model.fit(train_gen.flow(train_x, train_y), epochs=epochs,
                        validation_data=val_gen.flow(val_x, val_y))

    fig, axs = plt.subplots(1,2,figsize=(20,5))

    for i, metric in enumerate(['loss', 'accuracy']):
        axs[i].plot(history.history[metric])
        axs[i].plot(history.history['val_'+metric])
        axs[i].legend(['training', 'validation'], loc='best')

        axs[i].set_title('Model '+metric)
        axs[i].set_ylabel(metric)
        axs[i].set_xlabel('epoch')

    plt.show()

    print(f"Validation Accuracy: {model.evaluate(val_gen.flow(val_x, val_y))[1]}")

## Creating a basic CNN model

Below is a basic Convolutional Neural Network with 2 convolutional layers. The first layer has 32 filters and the second one has 64 filters. After each convolutional layer a max pooling layer is applied. Next, after the convolutional layers, a fully connected layer with 256 hidden nodes is applied, and finally a softmax layer with 10 outputs for each of the 10 classes.

Using the function from above, this model is then fitted on training data and evaluated on the validation data. You should get a validation accuracy of about *62%*. This means that already *62%* of the validation images is correctly classified as 1 of the 10 possible classes. Clearly, the network is learning something, as a random prediction for the validation set should result in about *10%* accuracy. Looking at the learning curves it also seems like the network is indeed learning something, although there is clearly room for improvement too.

Incrementally improving this network will be the core task of this assignment. For each subsequent improvement you should copy the network from the previous cell and make the new improvements in addition to the other improvements already made. You should of course start with the basic model provided here and the goal will be to improve the validation accuracy as much as possible.


In [ ]:
model = models.Sequential()
model.add(layers.Input(shape=(32, 32, 3)))

model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

train_and_evaluate(model, train_images, train_labels, test_images, test_labels)

Part of the assignment will also be to analyse the learning curves at each step to see if a specific improvement is likely to increase the validation accuracy. However, there is one important thing to note about these curves, which might be confusing otherwise. The training cost at each epoch is computed as the *average training cost* of all the batches in the epoch. This means for the first few batches the cost will probably be higher and will get lower with subsequent batches. The validation cost is computed after all the batches have been processed, meaning this cost can actually be *lower* than the training cost.

Normally, you would expect the training cost to always be lower than the validation cost, but because the training cost is computed as an average (which is more efficient) this might not be the case. The general shape of the training curve, especially for later iterations, will still be correct and so is still useful for analysis in the same way.

### Collaborative Questions

All open questions in this notebook are a little harder then the usual open questions, so for these you can, and should, confer with a fellow student on what you think the answers should be. As a result, similar answers will of course not be a problem for these question, just make sure to also mention with which students you discussed them.

**CQ1. Explain why you would normally expect the training cost to always be lower than the validation cost.**

*Your answer goes here.*

### Assignment 1: Normalizing the input data

The basic CNN model seems to clearly be overfitting, but before we address that, we'll start by normalizing the input. This is a first step that is important for many ML models, but specifically for deep neural networks, as ReLU activations are only actually non-linear around an input of 0 (where the input switches from 0 to a linear output). This means that, for the ReLU activations to really be effective, you would want the average combined inputs to end up around 0, which is why *mean centering the data* is so important.

You could use something like `np.mean` to compute the average input in all the training images and then subtract that average from every input, but the easiest way is to use the data preprocessor [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator). To use this together with the `train_and_evaluate()` function, you'll need to use the optional `preprocess` argument, which is a [keyword argument dictionary](https://docs.python.org/3/glossary.html#term-argument).

Each key in the *keyword argument dictionary* should be a string of the keyword you want to pass to the function and the value in the dictionary should be the corresponding parameter value you want to pass to the function. In order to mean center the data, we want to enable `featurewise_center`, so that would look like

    preprocess = {'featurewise_center': True}

Use this keyword argument dictionary to enable mean centering the data as a preprocessing step. In addition, the training data should be normalized to have a standard deviation of 1, for which there is also a keyword argument. The `train_and_evaluate()` function will then apply these same preprocessing steps to both the training and the validation data.

Start by copying the model from the cell before and then modify the `train_and_evaluate()` call to apply the mean centering and standard deviation normalization preprocessing to your data. If done correctly, your validation accuracy should increase to about *71%*.


**CQ2. Explain why normalizing your image features (i.e. pixels) to all have a standard deviation of 1 might also be important for deep neural networks with ReLU activations.**

*Your answer goes here.*

**CQ3. Explain why it is so important you apply the same normalization to your training and validation data. What would happen if the function did not do this?**

*Your answer goes here.*

In [ ]:
# YOUR CODE HERE


### Assignment 2: Adding Dropout

Looking at the learning curves from the previous cell, it seems like the model is definitely overfitting. An easy and effective way to address this can be adding dropout layers to the network. [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers are already included in `tensorflow` and all you need to specify for this layer is the probability for each input to be dropped (i.e. for the input to be set to 0).

Copy your code from the previous cell and add dropout layers to your network. You should add a Dropout layer before each of the 2 Dense layers, both with a probability of *0.3*, so approximately *30%* of the inputs of these layers will be set to 0! As you want to maintain more of the network activations closer to the network input, only add a Dropout layer before the *last* Conv2D layer, with a lower dropping probability of *0.2*. If done correctly, your validation accuracy should increase to about *75%*.


**CQ4. Explain what part exactly of the learning curves from the previous cell showed that the model was overfitting.**

*Your answer goes here.*

**CQ5. Explain in your own words why adding dropout reduces the chance of the model overfitting.**

*Your answer goes here.*

In [ ]:
# YOUR CODE HERE


Looking at the learning curves, the network also doesn't seem to be overfitting any more! An important thing to note about the training cost here is that it may again become higher than the validation cost. During training, dropout is active and some of the inputs get dropped, but when evaluating the validation data, all inputs are used, probably resulting in better prediction and thus a lower cost. To get a better estimate of the true training cost, we would need to add a full evaluation of the training data at each epoch too, but then we'd effectively double the computation for training data at each epoch, making the algorithm much slower. The training cost's current curve still has a useful shape for analysis, but the true training cost would be lower at every epoch.

### Assignment 3: Making the network deeper

Now that the network doesn't appear to be overfitting anymore, we can again try to add more complexity to the network and try to make it deeper. This should allow the network to learn more high level features and therefore hopefully improve performance further.

Start by copying the code from the previous cell and add 2 more Conv2D layers to the network; the first one with 128 filters and the second one with 256 filters. The new layers should be inserted after the existing convolutional layers, and both convolutional layers should have a Dropout layer *before*, using a dropping probability of 0.2, and a MaxPooling layer *after* (i.e. the same structure as the last convolutional layer).

As the network is now deeper, it will also take longer to train, so increase the number of epochs in `train_and_evalute()` to *50*. If all steps are done correctly, your validation accuracy should increase to about *79%*.


In [ ]:
# YOUR CODE HERE


As a last step, this code cell should print a summary of the `model` you've now constructed, with information about each of the added layers. Also run this cell and study the output produced there.

In [ ]:
model.summary()

**CQ6. Which layer has the largest total output volume (i.e. total number of nodes)? Explain why this layer has the most nodes.**

*Your answer goes here.*

**CQ7. Which layer has the largest number of trainable parameters  (i.e. weights)? Explain why this layer has the most weights.**

*Your answer goes here.*

### Assignment 4: Adding Batch Normalization

Now that the network is deeper, we might also benefit from applying batch normalization. With the preprocessing step at the start of this notebook the input features were normalized to have a 0 mean and standard deviation of 1, which helped the network learn mostly on the first layer with ReLU activations. With batch normalization this same type of scaling can be applied at each of the layers.

Start by copying the code from the previous cell and add a [BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) layer after each MaxPooling layer. Effectively, this places the BatchNormalization layers *before* each of the layers with ReLU activations, except for the input layer (which has already been normalized). If done correctly, your validation accuracy should increase to about 84%.


**CQ8. Explain in your own words why adding batch normalization improves the results of deep networks.**

*Your answer goes here.*

**CQ9. The BatchNormalization layer also has two learnable parameters, $\beta$ and $\gamma$, which start learning at 0 and 1 respectively. Based on the what these parameters do, you expect them to change a lot specifically when used just before a ReLU activation? Why or why not?**

*Your answer goes here.*

In [ ]:
# YOUR CODE HERE


### Assignment 5: Data Augmentation

The last change we'll try to make is using data augmentation, which artificially increases the size of the data by making variations in the training data. Augmentations on the data can be done using the [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator), which we used for preprocessing earlier.

The possible augmentations and the ranges for their values are listed in the documentation of [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator). Augmentations should be set in the `train_and_evaluate()` function using a separate *keyword argument dictionary*, which is called `augment`. For example, if you wanted to enable horizontal mirroring and a random zoom of maximum *10%* on training images, you would use

    augment = {'horizontal_flip': True, 'zoom_range': 0.1}

Try to find a combination of augmentations that improve the accuracy on the validation set even further. As the data set will now contain random variations, you'll probably have to increase the number of epochs quite a bit, depending on how many and how large a range of augmentations you make. This means training might actually take a long time, so it is not required to try enough combinations until you find a better validation accuracy, but only that you try and report your findings below.


In [ ]:
# YOUR CODE HERE


**CQ10. Explain why you would want to apply the augmentation to your training, but not your validation data. What would happen if the function did apply the augmentation to both?**

*Your answer goes here.*

**CQ11. Which augmentations did you try? Which augmentations gave the best results? Can you think of a reason why these augmentations specifically would be effective for the CIFAR-10 images?**

*Your answer goes here.*